# Procesamiento de los datos de Ecobici
## Equipo 3
### Integrantes: 
- Aponte Montes Carlos Daniel
- Cruz Ruiz Emiliano 
- Garcia Bernardino Daniel
- Larios Gutierrez Angela Atenea
- Rodríguez Almaraz Luis Antonio


In [64]:
from pyspark.sql.functions import col, sum, to_date, to_timestamp, regexp_replace, date_format 
from pyspark.sql.types import IntegerType, DoubleType, FloatType

In [65]:
dir_hdfs = "hdfs://namenode:9000/tmp/amd/ecobici_2012"

In [66]:
df = spark.read.csv(dir_hdfs, header=True, inferSchema=True)

In [67]:
df.printSchema()

root
 |-- Genero_Usuario: string (nullable = true)
 |-- Edad_Usuario: integer (nullable = true)
 |-- Bici: integer (nullable = true)
 |-- Ciclo_Estacion_Retiro: integer (nullable = true)
 |-- Fecha_Retiro: date (nullable = true)
 |-- Hora_Retiro: timestamp (nullable = true)
 |-- Ciclo_Estacion_Arribo: integer (nullable = true)
 |-- Fecha_Arribo: date (nullable = true)
 |-- Hora_Arribo: timestamp (nullable = true)



In [68]:
print(f"Columnas {df.columns}")

Columnas ['Genero_Usuario', 'Edad_Usuario', 'Bici', 'Ciclo_Estacion_Retiro', 'Fecha_Retiro', 'Hora_Retiro', 'Ciclo_Estacion_Arribo', 'Fecha_Arribo', 'Hora_Arribo']


In [69]:
df.show(truncate=False)

+--------------+------------+----+---------------------+------------+-----------------------+---------------------+------------+-------------------+
|Genero_Usuario|Edad_Usuario|Bici|Ciclo_Estacion_Retiro|Fecha_Retiro|Hora_Retiro            |Ciclo_Estacion_Arribo|Fecha_Arribo|Hora_Arribo        |
+--------------+------------+----+---------------------+------------+-----------------------+---------------------+------------+-------------------+
|M             |30          |1667|63                   |2012-11-01  |2024-10-20 00:00:10.72 |132                  |2012-11-01  |2024-10-20 00:21:23|
|M             |35          |2710|27                   |2012-11-01  |2024-10-20 00:01:10.847|128                  |2012-11-01  |2024-10-20 00:08:15|
|F             |45          |289 |67                   |2012-11-01  |2024-10-20 00:01:12.813|74                   |2012-11-01  |2024-10-20 00:04:04|
|M             |24          |1050|106                  |2012-11-01  |2024-10-20 00:01:43.017|106          

In [70]:
df = df.withColumn("Hora_Retiro", date_format(col("Hora_Retiro"), "HH:mm:ss"))\
       .withColumn("Hora_Arribo", date_format(col("Hora_Arribo"), "HH:mm:ss"))

df.show(truncate=False)


+--------------+------------+----+---------------------+------------+-----------+---------------------+------------+-----------+
|Genero_Usuario|Edad_Usuario|Bici|Ciclo_Estacion_Retiro|Fecha_Retiro|Hora_Retiro|Ciclo_Estacion_Arribo|Fecha_Arribo|Hora_Arribo|
+--------------+------------+----+---------------------+------------+-----------+---------------------+------------+-----------+
|M             |30          |1667|63                   |2012-11-01  |00:00:10   |132                  |2012-11-01  |00:21:23   |
|M             |35          |2710|27                   |2012-11-01  |00:01:10   |128                  |2012-11-01  |00:08:15   |
|F             |45          |289 |67                   |2012-11-01  |00:01:12   |74                   |2012-11-01  |00:04:04   |
|M             |24          |1050|106                  |2012-11-01  |00:01:43   |106                  |2012-11-01  |00:20:01   |
|M             |23          |2733|222                  |2012-11-01  |00:01:57   |54              

In [71]:
datos_retiro = df.select(['Genero_Usuario', 'Edad_Usuario','Bici', 'Ciclo_Estacion_Retiro', 'Fecha_Retiro', 'Hora_Retiro' ])

In [72]:
datos_arribo = df.select(['Genero_Usuario', 'Edad_Usuario','Bici', 'Ciclo_Estacion_Arribo', 'Fecha_Arribo', 'Hora_Arribo'])

In [73]:
datos_retiro.count()

2737917

In [74]:
#Casteo de columnas
#datos_retiro = datos_retiro.withColumn("Hora_Retiro", date_format(col("Hora_Retiro"), "HH:mm:ss"))

datos_retiro.show()
datos_retiro.printSchema()


+--------------+------------+----+---------------------+------------+-----------+
|Genero_Usuario|Edad_Usuario|Bici|Ciclo_Estacion_Retiro|Fecha_Retiro|Hora_Retiro|
+--------------+------------+----+---------------------+------------+-----------+
|             M|          30|1667|                   63|  2012-11-01|   00:00:10|
|             M|          35|2710|                   27|  2012-11-01|   00:01:10|
|             F|          45| 289|                   67|  2012-11-01|   00:01:12|
|             M|          24|1050|                  106|  2012-11-01|   00:01:43|
|             M|          23|2733|                  222|  2012-11-01|   00:01:57|
|             M|          25|1262|                   74|  2012-11-01|   00:03:34|
|             M|          38|1206|                  137|  2012-11-01|   00:04:30|
|             F|          42| 116|                   13|  2012-11-01|   00:04:32|
|             M|          29|1187|                   75|  2012-11-01|   00:05:38|
|             M|

In [75]:
#datos_arribo = datos_arribo.withColumn("Hora_Arribo", date_format(col("Hora_Arribo"), "HH:mm:ss"))

datos_arribo.show()
datos_arribo.printSchema()

+--------------+------------+----+---------------------+------------+-----------+
|Genero_Usuario|Edad_Usuario|Bici|Ciclo_Estacion_Arribo|Fecha_Arribo|Hora_Arribo|
+--------------+------------+----+---------------------+------------+-----------+
|             M|          30|1667|                  132|  2012-11-01|   00:21:23|
|             M|          35|2710|                  128|  2012-11-01|   00:08:15|
|             F|          45| 289|                   74|  2012-11-01|   00:04:04|
|             M|          24|1050|                  106|  2012-11-01|   00:20:01|
|             M|          23|2733|                   54|  2012-11-01|   00:21:33|
|             M|          25|1262|                   54|  2012-11-01|   00:08:42|
|             M|          38|1206|                  145|  2012-11-01|   00:09:59|
|             F|          42| 116|                  253|  2012-11-01|   00:08:48|
|             M|          29|1187|                   44|  2012-11-01|   00:12:27|
|             M|

## Desglose de tiempo
Por año (anio), mes, día, semana, hora, minuto y segundo para los tiempos y fechas de Retiro y Arribo

In [102]:
#Agregar campos anio, mes, dia, hora, minuto, segundo, dia de la semana en datos arribo
from pyspark.sql.functions import hour, minute, second, year, month, dayofmonth, weekofyear
colfs="Fecha_Retiro"
colhs = "Hora_Retiro"

df_tiempo_retiro = datos_retiro.select(
    col("Genero_Usuario"), col("Edad_Usuario"), col("Bici"), col("Ciclo_Estacion_Retiro"), 
    year(col(colfs)).alias("anio_retiro"),
    month(col(colfs)).alias("mes_retiro"),
    dayofmonth(col(colfs)).alias("dia_retiro"),
    weekofyear(col(colfs)).alias("semana_retiro"),
    date_format(col(colfs),"EEEE").alias("nombre_dia_retiro"),
    hour(col(colhs)).alias("hora_retiro"),
    minute(col(colhs)).alias("minuto_retiro"),
    second(col(colhs)).alias("segundo_retiro")



)

df_tiempo_retiro.show(truncate = False)

+--------------+------------+----+---------------------+-----------+----------+----------+-------------+-----------------+-----------+-------------+--------------+
|Genero_Usuario|Edad_Usuario|Bici|Ciclo_Estacion_Retiro|anio_retiro|mes_retiro|dia_retiro|semana_retiro|nombre_dia_retiro|hora_retiro|minuto_retiro|segundo_retiro|
+--------------+------------+----+---------------------+-----------+----------+----------+-------------+-----------------+-----------+-------------+--------------+
|M             |30          |1667|63                   |2012       |11        |1         |44           |Thursday         |0          |0            |10            |
|M             |35          |2710|27                   |2012       |11        |1         |44           |Thursday         |0          |1            |10            |
|F             |45          |289 |67                   |2012       |11        |1         |44           |Thursday         |0          |1            |12            |
|M             |

In [101]:
#Agregar campos anio, mes, dia, hora, minuto, segundo, dia de la semana en el datos_arribo
from pyspark.sql.functions import hour, minute, second, year, month, dayofmonth, weekofyear
colfs="Fecha_Arribo"
colhs = "Hora_Arribo"

df_tiempo_arribo = datos_arribo.select(
    col("Genero_Usuario"), col("Edad_Usuario"), col("Bici"), col("Ciclo_Estacion_Arribo"), 
    year(col(colfs)).alias("anio_arribo"),
    month(col(colfs)).alias("mes_arribo"),
    dayofmonth(col(colfs)).alias("dia_arribo"),
    weekofyear(col(colfs)).alias("semana_arribo"),
    date_format(col(colfs),"EEEE").alias("nombre_dia_arribo"),
    hour(col(colhs)).alias("hora_arribo"),
    minute(col(colhs)).alias("minuto_arribo"),
    second(col(colhs)).alias("segundo_arribo")



)

df_tiempo_arribo.show(truncate = False)

+--------------+------------+----+---------------------+-----------+----------+----------+-------------+-----------------+-----------+-------------+--------------+
|Genero_Usuario|Edad_Usuario|Bici|Ciclo_Estacion_Arribo|anio_arribo|mes_arribo|dia_arribo|semana_arribo|nombre_dia_arribo|hora_arribo|minuto_arribo|segundo_arribo|
+--------------+------------+----+---------------------+-----------+----------+----------+-------------+-----------------+-----------+-------------+--------------+
|M             |30          |1667|132                  |2012       |11        |1         |44           |Thursday         |0          |21           |23            |
|M             |35          |2710|128                  |2012       |11        |1         |44           |Thursday         |0          |8            |15            |
|F             |45          |289 |74                   |2012       |11        |1         |44           |Thursday         |0          |4            |4             |
|M             |

## Guardar / crear df temporal

In [78]:
#Guardar el nuevo df

In [103]:
df.createOrReplaceTempView("ECOBICI")
df_tiempo_retiro.createOrReplaceTempView("ECOBICI_retiro")
df_tiempo_arribo.createOrReplaceTempView("ECOBICI_arribo")


In [104]:
spark.sql("show tables").show()

+---------+--------------+-----------+
|namespace|     tableName|isTemporary|
+---------+--------------+-----------+
|         |       ecobici|       true|
|         |ecobici_arribo|       true|
|         |ecobici_retiro|       true|
+---------+--------------+-----------+



In [81]:
spark.sql("describe ecobici").show()

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|      Genero_Usuario|   string|   NULL|
|        Edad_Usuario|      int|   NULL|
|                Bici|      int|   NULL|
|Ciclo_Estacion_Re...|      int|   NULL|
|        Fecha_Retiro|     date|   NULL|
|         Hora_Retiro|   string|   NULL|
|Ciclo_Estacion_Ar...|      int|   NULL|
|        Fecha_Arribo|     date|   NULL|
|         Hora_Arribo|   string|   NULL|
+--------------------+---------+-------+



## 10 estaciones con mayores viajes de inicio

In [83]:
#Mostrar las 10 estaciones con mayores viajes de inicio
strsql = """
SELECT Ciclo_Estacion_Retiro AS estacion_inicio, COUNT(*) AS num_viajes
FROM ecobici
GROUP BY Ciclo_Estacion_Retiro
ORDER BY num_viajes DESC
LIMIT 10
"""
spark.sql(strsql).show(10, truncate=False)

+---------------+----------+
|estacion_inicio|num_viajes|
+---------------+----------+
|36             |58881     |
|27             |56857     |
|1              |55942     |
|64             |54646     |
|41             |52026     |
|13             |45488     |
|85             |44145     |
|18             |43521     |
|84             |42217     |
|21             |39311     |
+---------------+----------+



## Las 5 horas con mayor demanda

In [84]:
#Mostrar las 5 horas con mayor demanda
strsql = """
SELECT Hora_Retiro AS hora, COUNT(*) AS num_viajes
FROM ecobici
GROUP BY Hora_Retiro
ORDER BY num_viajes DESC
LIMIT 5
"""
spark.sql(strsql).show(5, truncate=False)

+--------+----------+
|hora    |num_viajes|
+--------+----------+
|08:48:55|113       |
|08:57:33|112       |
|08:55:28|110       |
|08:54:08|110       |
|08:57:36|109       |
+--------+----------+



In [87]:
#Mostrar las 5 horas con mayor demanda
strsql = """
SELECT HOUR(Hora_Retiro) AS hora, COUNT(*) AS num_viajes
FROM ecobici
GROUP BY HOUR(Hora_Retiro)
ORDER BY num_viajes DESC
LIMIT 5
"""
spark.sql(strsql).show(5, truncate=False)

+----+----------+
|hora|num_viajes|
+----+----------+
|18  |234658    |
|8   |222619    |
|14  |220045    |
|19  |217117    |
|15  |210520    |
+----+----------+



## El día de la semana con menor demanda

In [91]:
#Mostrar el dia de la semana con menor demanda
strsql = """
SELECT DAYOFWEEK(Fecha_Retiro) AS dia_semana, COUNT(*) AS num_viajes
FROM ecobici
GROUP BY DAYOFWEEK(Fecha_Retiro)
ORDER BY num_viajes ASC
LIMIT 1
"""
spark.sql(strsql).show(truncate=False)

+----------+----------+
|dia_semana|num_viajes|
+----------+----------+
|1         |201247    |
+----------+----------+



In [105]:
strsql = """
SELECT nombre_dia_retiro AS dia_semana, COUNT(*) AS num_viajes
FROM ecobici_retiro
GROUP BY nombre_dia_retiro
ORDER BY num_viajes ASC
LIMIT 1
"""
spark.sql(strsql).show(truncate=False)

+----------+----------+
|dia_semana|num_viajes|
+----------+----------+
|Sunday    |201247    |
+----------+----------+



## Estadísticas descriptivas

In [97]:
from pyspark.sql import functions as F

stats = spark.table("ecobici").agg(
    F.count("*").alias("Total"),
    F.mean("Edad_Usuario").alias("Promedio de edad"),
    F.min("Edad_Usuario").alias("Minimo de edad"),
    F.max("Edad_Usuario").alias("Maximo de edad"),
    F.stddev("Edad_Usuario").alias("Desviación estandar")
)

stats.show()


+-------+-----------------+--------------+--------------+-------------------+
|  Total| Promedio de edad|Minimo de edad|Maximo de edad|Desviación estandar|
+-------+-----------------+--------------+--------------+-------------------+
|2737917|34.98756719067817|            16|            80| 10.167090270059765|
+-------+-----------------+--------------+--------------+-------------------+



In [95]:
from pyspark.sql import functions as F


stats_genero = spark.table("ecobici").groupBy("Genero_Usuario").agg(
    F.count("*").alias("Total_Viajes"),
    F.countDistinct("Edad_Usuario").alias("Total_Edades_Distintas"),
    F.mean("Edad_Usuario").alias("Edad_Promedio"),
    F.min("Edad_Usuario").alias("Edad_Minima"),
    F.max("Edad_Usuario").alias("Edad_Maxima"),
    F.stddev("Edad_Usuario").alias("Edad_Desviacion_Estandar")
)

stats_genero.show(truncate=False)

+--------------+------------+----------------------+------------------+-----------+-----------+------------------------+
|Genero_Usuario|Total_Viajes|Total_Edades_Distintas|Edad_Promedio     |Edad_Minima|Edad_Maxima|Edad_Desviacion_Estandar|
+--------------+------------+----------------------+------------------+-----------+-----------+------------------------+
|F             |777068      |63                    |33.334058280613796|16         |80         |9.312471175197944       |
|M             |1960849     |65                    |35.642838892744926|16         |80         |10.414134576273897      |
+--------------+------------+----------------------+------------------+-----------+-----------+------------------------+



In [99]:
from pyspark.sql import functions as F

stats_bici = spark.table("ecobici").agg(
    F.countDistinct("Bici").alias("Total_Bicis_Distintas"),
)

stats_bici.show(truncate=False)

+---------------------+
|Total_Bicis_Distintas|
+---------------------+
|3358                 |
+---------------------+



## Agrupaciones y Gráficas

### Viajes por estación

In [ ]:
#Ejecuta y luego borra las celdas
!pip install matplotlib

In [ ]:
!pip install numpy

In [ ]:
viajes_por_estacion = df.groupBy('Ciclo_Estacion_Retiro').\
    count().orderBy('Ciclo_Estacion_Retiro', ascending=True)
print(f"Viajes por estación")


In [ ]:
import matplotlib.pyplot as plt

# Obtener los datos de viajes por estación
viajes_por_estacion_data = viajes_por_estacion.collect()

# Separar los datos en listas para la gráfica
estaciones = [row.Ciclo_Estacion_Retiro for row in viajes_por_estacion_data]
cuentas = [row['count'] for row in viajes_por_estacion_data]

# Crear la gráfica
plt.figure(figsize=(12, 6))
plt.bar(estaciones, cuentas, color='#c74777')
plt.xlabel('Estación Retiro')
plt.ylabel('Número de Viajes')
plt.title('Viajes por Estación')
plt.xticks(rotation=45)
plt.tight_layout()  

# Mostrar la gráfica
viajes_por_estacion_data.show(100, truncate=False)
plt.show()

### Viajes por Género

In [ ]:
viajes_por_genero = df.groupBy('Genero_Usuario').count().orderBy('count', ascending=True)

print(f"Viajes por Género")

In [ ]:
import matplotlib.pyplot as plt

# Obtener los datos de viajes por género
viajes_por_genero_data = viajes_por_genero.collect()

# Separar los datos en listas para la gráfica
generos = [row.Genero_Usuario for row in viajes_por_genero_data]
cuentas_genero = [row['count'] for row in viajes_por_genero_data]

# Crear la gráfica
plt.figure(figsize=(8, 5))
plt.bar(generos, cuentas_genero, color='#e98dad')
plt.xlabel('Género del Usuario')
plt.ylabel('Número de Viajes')
plt.title('Viajes por Género en Noviembre')
plt.xticks(rotation=45)
plt.tight_layout()  

# Mostrar la gráfica
viajes_por_genero.show()
plt.show()

### Viajes por edad

In [ ]:
viajes_por_edad = df.groupBy('Edad_Usuario').count().orderBy('Edad_Usuario')

# Mostrar resultados
print("Viajes por edad de usuario")

In [ ]:
import matplotlib.pyplot as plt

# Obtener los datos de viajes por edad
viajes_por_edad_data = viajes_por_edad.collect()

# Separar los datos en listas para la gráfica
edades = [row.Edad_Usuario for row in viajes_por_edad_data]
cuentas_edad = [row['count'] for row in viajes_por_edad_data]

# Crear la gráfica
plt.figure(figsize=(10, 6))
plt.bar(edades, cuentas_edad, color='#c74777')
plt.xlabel('Edad del Usuario')
plt.ylabel('Número de Viajes')
plt.title('Viajes por Edad de Usuario')
plt.xticks(rotation=45)
plt.tight_layout()  

# Mostrar la gráfica
viajes_por_edad.show(100, truncate=False)
plt.show()

### Viajes por edad y género

In [ ]:
viajes_por_edad_genero = df.groupBy('Edad_Usuario', 'Genero_Usuario').count().orderBy('Edad_Usuario')

# Mostrar resultados
print("Viajes por edad y género")

In [ ]:
from pyspark.sql.functions import col, floor

# Agrupar edades en intervalos de 10
df = df.withColumn(
    'Grupo_Edad', (floor(col('Edad_Usuario') / 10) * 10)
)

# Agrupar por el nuevo grupo de edad y género
viajes_por_edad_genero = df.groupBy('Grupo_Edad', 'Genero_Usuario').count().orderBy('Grupo_Edad')

# Mostrar resultados
print("Viajes por grupo de edad y género")
viajes_por_edad_genero.show(100, truncate=False)

# Obtener los datos de viajes por grupo de edad y género
viajes_por_edad_genero_data = viajes_por_edad_genero.collect()

# Separar los datos en listas
grupos_edad = sorted(set(row.Grupo_Edad for row in viajes_por_edad_genero_data))
generos = sorted(set(row.Genero_Usuario for row in viajes_por_edad_genero_data))

# Diccionario para contar los viajes por grupo de edad y género
contar_viajes = {grupo: {genero: 0 for genero in generos} for grupo in grupos_edad}

# Diccionario con los datos
for row in viajes_por_edad_genero_data:
    contar_viajes[row.Grupo_Edad][row.Genero_Usuario] += row['count']

# Listas para los valores de la gráfica
valores_hombres = [contar_viajes[grupo]['M'] for grupo in grupos_edad]
valores_mujeres = [contar_viajes[grupo]['F'] for grupo in grupos_edad]

# Crear la gráfica de líneas
plt.figure(figsize=(12, 6))
plt.plot(grupos_edad, valores_hombres, marker='o', label='Hombres', color='#4f021b', linestyle='-')
plt.plot(grupos_edad, valores_mujeres, marker='o', label='Mujeres', color='#c74777', linestyle='-')

# Etiquetas y título
plt.xlabel('Grupo de Edad')
plt.ylabel('Número de Viajes')
plt.title('Viajes por Grupo de Edad y Género en Septiembre')
plt.xticks(grupos_edad)
plt.legend()
plt.tight_layout()

# Mostrar la gráfica
viajes_por_edad_genero.show(100, truncate=False)
plt.show()

### Viajes por semana y estación de retiro

In [ ]:
from pyspark.sql.functions import date_format, to_date, col

df = df.withColumn('Fecha_Retiro', to_date(col('Fecha_Retiro')))
df = df.withColumn('Fecha_Arribo', to_date(col('Fecha_Arribo')))

# Agrupar por fecha de retiro, fecha de arribo y estación de retiro
viajes_por_dia_y_estacion = df.groupBy(
    'Fecha_Retiro', 'Fecha_Arribo', 'Ciclo_Estacion_Retiro'
).count().orderBy('Fecha_Arribo')

In [ ]:
from pyspark.sql import functions as F
import matplotlib.pyplot as plt

# 5 estaciones principales por número de viajes
top_estaciones = viajes_por_dia_y_estacion.groupBy('Ciclo_Estacion_Retiro').agg(F.sum('count').alias('total')).orderBy('total', ascending=False).limit(5)

# Lista de estaciones 
top_estaciones_list = [row.Ciclo_Estacion_Retiro for row in top_estaciones.collect()]

# Agrupar estaciones principales por semana
viajes_por_semana_y_estacion = df.groupBy(
    date_trunc('week', col('Fecha_Retiro')).alias('Semana_Retiro'),
    'Ciclo_Estacion_Retiro'
).agg(F.count('*').alias('count')).filter(col('Ciclo_Estacion_Retiro').isin(top_estaciones_list)).orderBy('Semana_Retiro')

# Mostrar resultados
print("Viajes por día y estación de retiro y arribo")
viajes_por_semana_y_estacion.show(100,truncate=False)

viajes_por_semana_y_estacion_data = viajes_por_semana_y_estacion.collect()

# Listas para la gráfica
semanas = sorted(set(row.Semana_Retiro for row in viajes_por_semana_y_estacion_data))
estaciones = top_estaciones_list  # Usar solo las estaciones principales

# Diccionario para contar los viajes por semana y estación
contar_viajes = {semana: {estacion: 0 for estacion in estaciones} for semana in semanas}

# Diccionario con los datos
for row in viajes_por_semana_y_estacion_data:
    if row.Ciclo_Estacion_Retiro in contar_viajes[row.Semana_Retiro]:  # Verificar si la estación está en el diccionario
        contar_viajes[row.Semana_Retiro][row.Ciclo_Estacion_Retiro] += row['count']

# Listas para los valores de la gráfica
valores_estaciones = {estacion: [contar_viajes[semana][estacion] for semana in semanas] for estacion in estaciones}

# Gráfica de líneas
plt.figure(figsize=(12, 6))
colores=['#e3558d','#e98dad','#c74777','#7a042a','4f021b']

# Grafica de cada estación
for i, (estacion, valores) in enumerate(valores_estaciones.items()):
    plt.plot(semanas, valores, marker='o', label=estacion, color=colores[i % len(colores)])

# Etiquetas y título
plt.xlabel('Semana')
plt.ylabel('Número de Viajes')
plt.title('Viajes por Semana y Estación de Retiro (Top 5)')
plt.xticks(rotation=45)
plt.legend()

# Mostrar la gráfica
plt.tight_layout()
plt.show()

